In [14]:
import ee
import pandas as pd
import time

# Initialize the Earth Engine module.
ee.Initialize()

df = pd.read_csv('../data/label.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13145 entries, 0 to 13144
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 13145 non-null  int64  
 1   cell_id              13145 non-null  object 
 2   centroid_longtitude  13145 non-null  float64
 3   centroid_latitude    13145 non-null  float64
 4   wi                   13145 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 513.6+ KB


In [16]:
PREFIX = 'asset-index-2'
SCALE = 30  # meters
KSIZE = 112  # half the size of the desired patch in pixels (225 / 2)
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11','B12']
SCALE_FACTOR = 10000  # Scaling factor for Sentinel-2 reflectance values
DATASET = 'COPERNICUS/S2'
NUM_BATCH = 20
BATCH_SIZE = len(df) // NUM_BATCH

def get_quarter_dates(year):
    # Returns a list of date ranges for each quarter
    return [
        (f'{year}-01-01', f'{year}-03-31'), # Q1
        (f'{year}-04-01', f'{year}-06-30'), # Q2
        (f'{year}-07-01', f'{year}-09-30'), # Q3
        (f'{year}-10-01', f'{year}-12-31')  # Q4
    ]

def export_image(longtitude, latitude, year, wi):

    # Define the point of interest and create a buffer region around it
    point = ee.Geometry.Point([longtitude, latitude])
    buffered_point = point.buffer(SCALE * KSIZE)

    # Function to estimate cloud coverage over the ROI
    def cloudiness(image):
        qa_band = image.select('QA60')
        opaque_clouds = qa_band.bitwiseAnd(1 << 10).neq(0)
        cirrus_clouds = qa_band.bitwiseAnd(1 << 11).neq(0)
        total_cloud_mask = opaque_clouds.Or(cirrus_clouds)
        cloud_coverage_ratio = total_cloud_mask.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffered_point,
            scale=SCALE
        ).get('QA60')
        return image.set('cloudiness', cloud_coverage_ratio)
    
    quarterly_images = []

     # Retrieve the least cloudy image of the year as a fallback
    yearly_collection = ee.ImageCollection(DATASET) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .filterBounds(buffered_point) \
        .map(cloudiness)

    yearly_least_cloudy = yearly_collection.sort('cloudiness').first().divide(SCALE_FACTOR)

    
    for i, (start_date, end_date) in enumerate(get_quarter_dates(year)):
        collection = ee.ImageCollection(DATASET) \
            .filterDate(start_date, end_date) \
            .filterBounds(buffered_point) \
            .map(cloudiness)
        
        # Check if the collection is empty
        count = collection.size().getInfo()
        if count == 0:
            least_cloudy_image = yearly_least_cloudy
        else:
            least_cloudy_image = collection.sort('cloudiness').first().divide(SCALE_FACTOR)

        quarter_suffix = f'_{i+1}'
        renamed_bands = [band + quarter_suffix for band in BANDS]
        least_cloudy_image = least_cloudy_image.select(BANDS, renamed_bands)


        quarterly_images.append(least_cloudy_image)

    # Combine the quarterly images into a single image
    combined_image = ee.Image.cat(quarterly_images)
    
    # Define the point of interest as a feature collection
    points = ee.FeatureCollection([ee.Feature(point, {'longtitude': longtitude, 'latitude': latitude})])

        # Set up the export task
    task = get_array_patches(
        combined_image,
        points,
        longtitude,
        latitude,
        year,
        wi
    )
    # Start the export
    task.start()

# Function to get array patches
def get_array_patches(img, points, longtitude, latitude, year, wi):
    kern = ee.Kernel.square(radius=KSIZE, units='pixels')
    patches_array = img.neighborhoodToArray(kern)
    samples = points.map(lambda pt: sample_patch(pt, patches_array, longtitude, latitude, wi))

    # Export to TFRecord file
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=PREFIX,
        folder=PREFIX,  
        fileNamePrefix=f'{year}_{longtitude}_{latitude}_{wi}',
        fileFormat='TFRecord'
    )
    return task

# Function to sample patch and include additional properties
def sample_patch(point, patches_array, longtitude, latitude, wi):
    arrays_samples = patches_array.sample(
        region=point.geometry(),
        scale=SCALE,
        projection='EPSG:3857',
        factor=None,
        numPixels=None,
        dropNulls=False,
        tileScale=12)
    sample_feature = arrays_samples.first()

    # Set additional properties to the feature
    sample_feature = sample_feature.set({
        'longtitude': longtitude,
        'latitude': latitude,
        'wi' : wi
    })
    return sample_feature

In [17]:
for i in range(NUM_BATCH):
    batch_df = df[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
    print(i)
    if i < 12: 
        continue
    for idx, row in batch_df.iterrows():
        longtitude = row['centroid_longtitude']
        latitude = row['centroid_latitude']
        year = row['year']
        wi = row['wi']
        try:
            export_image(longtitude, latitude, 2016 if year == 2014 else year, wi)
        except:
            print(f'Error at {longtitude}, {latitude}, {year}')
            time.sleep(30)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
